# Building the AskReddit Corpus about Covid

**I'm using this 2 datasets from kaggle where they collected a good number of Askreddit Posts.  
With a a little processing we can make a hopefully usefull corpus.**

[URL donde correr la notebook](https://www.kaggle.com/mauroebordon/0-askreddit-corpus)

In [ ]:
!pip install -U pandas
import re
import pandas as pd

In [ ]:
#Creamos el primer Dataframe con el primer Dataset

#Está separado en 2 archivos (El path es el default de Kaggle, recomiendo correr ahi o tendran q modificar esto)
qs_df1 = pd.read_csv("../input/askreddit-questions-and-answers/reddit_questions.csv", delimiter=';')
ans_df1 = pd.read_csv("../input/askreddit-questions-and-answers/reddit_answers.csv", delimiter=';')

#Merge y lo limpiamos un poquito
df1 = qs_df1.merge(ans_df1, left_on="id", right_on="q_id").drop(columns=["datetime", "q_id", "timestamp"]).rename(columns={"text_x": "Q", "text_y": "ANS", "votes_x": "Qscore", "votes_y": "ANSscore"}).drop(columns=["Unnamed: 0"])

#Temporal: solo dejamos los comentarios con más upvotes
df1 = df1.groupby("id", as_index=False).agg(lambda x: max(x))

df1

In [ ]:
#Creamos el segundo dataframe para el otro dataset

#Está separado en 2 archivos
ans_df2 = pd.read_csv("../input/a-month-of-askreddit/askreddit_comments.csv", dtype=str).drop(columns=[ "permalink", "timestamp", "author"])
qs_df2 = pd.read_csv("../input/a-month-of-askreddit/askreddit_posts.csv", dtype=str).drop(columns=["author", "timestamp", "num_comments", "selftext", "permalink"]).rename({"title": "text"})

#filtramos los comentarios que sean directos al post.
ans_df2 = ans_df2[ans_df2["post_id"] == ans_df2["parent_id"]]

#Normalizamos el id
ans_df2["post_id"] = ans_df2["post_id"].str.replace("t3_", "")

ans_df2 = ans_df2.drop(columns=[ "parent_id"]).rename(columns={"body": "ANS", "score": "ANSscore", "post_id": "id"})

#Ajustamos los nombres para que coincidan
qs_df2 = qs_df2.rename(columns={"post_id": "id", "title": "Q", "score": "Qscore"})

df2 = qs_df2.merge(ans_df2, left_on="id", right_on="id")


df2 = df2.groupby("id", as_index=False).first()

df2

In [ ]:
# Los concatenamos, Y Ya tenemos un lindo corpus de AskReddit para multiple uso
df = pd.concat([df1, df2], ignore_index=True)

#lo guardamos depaso
df.to_csv("ask-reddit.csv")
df

In [ ]:


#qs1_df[qs1_df["text"].str.contains(r"deworm", flags=re.IGNORECASE)]

#qs1_df[qs1_df["text"].str.contains(r"conspiracy|hoax|tested|pfizer|covid|vax|Sputnik|Moderna|Jab|vacc|corona|virus|Dosis|Cases|Face\W?Mask|pandemic|quarantine|social distanc", flags=re.IGNORECASE)]